In [1]:
import girder_client
import pandas as pd
import numpy as np
import json

In [2]:
gc = girder_client.GirderClient(apiUrl='https://isic-archive.com/api/v1')
gc.authenticate('dominikkiefer', 'dlproject')

In [3]:
pd.DataFrame(gc.listCollection())

,_accessLevel,_id,_modelType,created,description,name,public,size,updated
0,-1,55943cff9fc3c13155bcad61,collection,2015-07-01T19:18:23.988000+00:00,Clinical feature annotation studies,Annotation Studies,True,0,2016-08-24T03:06:20.532000+00:00
1,-1,55943cff9fc3c13155bcad5e,collection,2014-09-29T02:59:17.932000+00:00,All public lesion image datasets,Lesion Images,True,293175840740,2016-08-24T03:07:21.780000+00:00


In [4]:
#gc.downloadResource("55943cff9fc3c13155bcad5e",dest="/home/dwx/tmp")

dataset_df = pd.DataFrame(gc.get("/dataset"))
dataset_df

,_id,name,updated
0,5627f42b9fc3c132be08d84f,ISIC_MSK-1_1,2015-10-21T20:33:21.466000+00:00
1,582b8d419fc3c1566bbcb098,ISIC_MSK-1_2,2017-04-13T20:08:46.159000+00:00
2,5627f5f69fc3c132be08d852,ISIC_MSK-2_1,2015-10-21T20:33:29.916000+00:00
3,57eebe389fc3c12a89bb75f7,ISIC_MSK-3_1,2017-04-13T20:07:38.035000+00:00
4,581cd6059fc3c13dcd0e0930,ISIC_MSK-4_1,2017-04-13T20:08:05.005000+00:00
5,5825fd959fc3c171066d3352,ISIC_MSK-5_1,2017-04-13T20:08:24.725000+00:00
6,5627eefe9fc3c132be08d84c,ISIC_SONIC_1,2016-08-10T15:36:43.866000+00:00
7,54b6e869bae4785ee2be8652,ISIC_UDA-1_1,2014-11-10T02:39:56.492000+00:00
8,54ea816fbae47871b5e00c80,ISIC_UDA-2_1,2015-03-06T14:54:30.984000+00:00


In [44]:
dataset_names = list(dataset_df["name"])
dataset_names

['ISIC_MSK-1_1',
 'ISIC_MSK-1_2',
 'ISIC_MSK-2_1',
 'ISIC_MSK-3_1',
 'ISIC_MSK-4_1',
 'ISIC_MSK-5_1',
 'ISIC_SONIC_1',
 'ISIC_UDA-1_1',
 'ISIC_UDA-2_1']

In [58]:
db_info = []
for a in dataset_names[:]:
    cur_dataset_id = dataset_df[dataset_df["name"] == a]._id
    cur_dataset_id = cur_dataset_id.tolist()[0]
    path = "/dataset/"+str(cur_dataset_id)
    db_info += [gc.get(path)]

db_info = pd.DataFrame(db_info)

In [60]:
path_dest = "/home/dwx/tmp/"
db_info.to_csv(path_dest+"db_info.csv")

#### Get Image Metadata

In [63]:
def getSpecificMetadata(jsonData):
    #result += [jsonData["meta"]["clinical"]["benign_malignant"]]
    result = jsonData["meta"]["clinical"]
    result["name"] = jsonData["name"]
    return result

In [84]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
path_image = "/image/"
path_dest = "/home/dwx/tmp/"

for a in dataset_names[:]:
    cur_dataset_id = dataset_df[dataset_df["name"] == a]._id
    cur_dataset_id = cur_dataset_id.tolist()[0]
    
    cur_selection = gc.get(path_image,parameters={"datasetId":cur_dataset_id, "limit":2000})
    result = []
    for b in log_progress(cur_selection):
        cur_id = b["_id"]
        curMetadata = gc.get(path_image+str(curId),parameters={})
        specificMetadata = getSpecificMetadata(curMetadata)
        result += [specificMetadata]
    result_df = pd.DataFrame(result)
    result_df.to_csv(path_dest+a+".csv")

In [9]:
#dir(curSelection)